This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace.

## SETUP

In [3]:
# This command installs the following libraries for a Large Language Model (LLM) or AI-powered application:
# - lancedb: A fast, lightweight vector database for storing and querying embeddings.
# - langchain==0.0.305: A framework that streamlines the development of LLM-driven workflows (prompt engineering, chaining, etc.).
# - openai==0.28.1: The official OpenAI Python library for accessing models like GPT via the OpenAI API.
# - torch: PyTorch, a deep learning framework used for model training and inference.
# - chromadb: Another vector database often used for similarity search and storing embeddings.
!pip install -U lancedb langchain==0.0.305 openai==0.28.1 torch chromadb

INFO: pip is looking at multiple versions of lancedb to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━

In [4]:
# lancedb: A fast, lightweight vector database for storing and querying embeddings.
import lancedb

# LanceModel, vector: Pydantic models/extensions tailored for defining and validating LanceDB schemas.
from lancedb.pydantic import LanceModel, vector

# shutil: Provides high-level file operations (copy, move, etc.).
import shutil

# pandas: A data analysis library for managing structured data (e.g., CSV files).
import pandas as pd

# OpenAI: LangChain’s interface for interacting with OpenAI LLMs.
from langchain.llms import OpenAI

# openai: The official OpenAI library for making calls to OpenAI’s API.
import openai

# PromptTemplate, FewShotPromptTemplate: Classes for creating and managing prompt templates in LangChain.
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# PydanticOutputParser: Helps parse AI model outputs into structured data using Pydantic.
from langchain.output_parsers import PydanticOutputParser

# BaseModel, Field, NonNegativeInt: Pydantic features for model validation and data type enforcement.
from langchain_core.pydantic_v1 import BaseModel, Field, NonNegativeInt

# OpenAIEmbeddings: Generates embeddings using OpenAI’s Embeddings API.
from langchain.embeddings import OpenAIEmbeddings

# Document: A standardized structure in LangChain for handling textual data.
from langchain.schema import Document

# RecursiveCharacterTextSplitter: Utility to split large texts into smaller chunks for LLM processing.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Chroma: A vector store used to index and search embeddings efficiently.
from langchain.vectorstores.chroma import Chroma

# CSVLoader: Loads CSV data and converts rows into Documents.
from langchain.document_loaders import CSVLoader

# List: Generic typing support for function signatures and type hints.
from typing import List

# (Repeated) PromptTemplate: Used to build reusable prompt templates in LangChain.
from langchain.prompts import PromptTemplate

# DiffusionPipeline, AutoPipelineForText2Image: Pipelines from Diffusers for text-to-image generation.
from diffusers import DiffusionPipeline, AutoPipelineForText2Image

# load_image, make_image_grid: Utility methods for image handling and creating image grids.
from diffusers.utils import load_image, make_image_grid

# jsonable_encoder: Utility to convert Python objects to JSON-serializable format (commonly used in FastAPI).
from fastapi.encoders import jsonable_encoder

# PIL: Python Imaging Library for image manipulation.
import PIL

# torch: PyTorch, a deep learning framework for building and training neural networks.
import torch

# sys: Provides system-specific parameters and functions (e.g., sys.path).
import sys

# os: Operating system interfaces (e.g., for environment variables and directory manipulation).
import os


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
# Setting environment variables for the OpenAI API:
# - OPENAI_API_KEY: Your API key for authenticating requests to OpenAI.
# - OPENAI_API_BASE: The base URL for sending API calls (e.g., a custom or proxy endpoint).
# Note: In production settings, do not commit your API keys to version control.
#       Instead, store them securely (e.g., environment variables, vault, etc.).
os.environ["OPENAI_API_KEY"] = "" # Hidden for security purpose
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"


In [8]:
# Assign your OpenAI API key from the environment variable.
# This authenticates your requests to OpenAI.
openai.api_key = os.environ["OPENAI_API_KEY"]

# Set the base URL for the OpenAI API from the environment variable.
# This can be used to point to a custom or proxy endpoint.
openai.api_base = os.environ["OPENAI_API_BASE"]

## Real Estate Listing Generation

In [9]:
# INSTRUCTION holds the prompt that tells the model to generate
# 10 different real estate listings for various Paris neighborhoods.
INSTRUCTION = "Generate 10 different and realistic real estate listings from multiple Neighborhoods of Paris."

# SAMPLE_LISTING provides an example of the desired listing format and details,
# including the neighborhood, price, and a brief description of the property.
# This helps guide the model on structure and content style.
SAMPLE_LISTING = \
"""
A sample listing is the following as shown below:

"Neighborhood": "Paris 4th District"
"Price (USD)": 773515
"Bedrooms": 3
"Bathrooms": 2
"House Size sqft": 549
"Description": "This south-facing 51 sqm split-level apartment is on the 1st floor of a late 18th century building. Overlooking a peaceful leafy courtyard, it comprises a spacious living room and dining room, and an equipped open-plan kitchen. A bedroom with a shower room and wc is upstairs."
"Neighborhood description": "Rue de Jarente, near Place du March Sainte Catherine in the heart of the historic Marais neighbourhood."

"""


In [10]:
# This Pydantic model represents a single real estate listing,
# defining fields for neighborhood, price, and more.
# Each field has a description to guide validation and documentation.
class REListing(BaseModel):
    neighborhood: str = Field(description="Name of the neighborhood")
    price: NonNegativeInt = Field(description="Price of the property in USD")
    bedrooms: NonNegativeInt = Field(description="Number of bedrooms in the property")
    bathrooms: NonNegativeInt = Field(description="Number of bathrooms in the property")
    house_size: NonNegativeInt = Field(description="Size of the property in square feet")
    description: str = Field(description="Description of the property.")
    neighborhood_description: str = Field(description="A description of the neighborhood.")

# This Pydantic model serves as a container for multiple real estate listings.
class REListingCollection(BaseModel):
    listings: List[REListing] = Field(description="List of available real estate listings")

# The parser uses PydanticOutputParser to take raw text (from an LLM or other source)
# and parse it into the REListingCollection model defined above.
parser = PydanticOutputParser(pydantic_object=REListingCollection)

In [11]:
# The PromptTemplate configures how instructions, samples, and format instructions
# are combined into a prompt for the language model. The 'partial_variables' parameter
# injects the parser’s formatting instructions automatically.
prompt = PromptTemplate(
    template="{instruction}\n{sample}\n{format_instructions}\n",
    input_variables=["instruction", "sample"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

# This generates the final prompt by filling in the placeholders
# with the instruction and sample listing, then prints it for debugging or review.
query = prompt.format(instruction=INSTRUCTION, sample=SAMPLE_LISTING)
print(query)

Generate 10 different and realistic real estate listings from multiple Neighborhoods of Paris.

A sample listing is the following as shown below:

"Neighborhood": "Paris 4th District"
"Price (USD)": 773515
"Bedrooms": 3
"Bathrooms": 2
"House Size sqft": 549
"Description": "This south-facing 51 sqm split-level apartment is on the 1st floor of a late 18th century building. Overlooking a peaceful leafy courtyard, it comprises a spacious living room and dining room, and an equipped open-plan kitchen. A bedroom with a shower room and wc is upstairs."
"Neighborhood description": "Rue de Jarente, near Place du March Sainte Catherine in the heart of the historic Marais neighbourhood."


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-f

In [12]:
# Define the model configuration for OpenAI's GPT:
# - model_name: The name of the GPT model (e.g., gpt-3.5-turbo).
# - temperature: Controls the creativity/randomeness of the output.
#   A higher temperature (e.g., 1.0) results in more varied responses,
#   while a lower temperature (e.g., 0.0) makes the responses more deterministic.
# - max_tokens: The maximum number of tokens (words/pieces) in the response.
model_name = "gpt-3.5-turbo"
temperature = 1.0
max_tokens = 3500

# Instantiate the LLM object with the specified parameters.
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens=max_tokens)

# Get the response to the query using the configured LLM.
response = llm(query)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [13]:
# Parse the LLM's response into the structured data model using the parser.
result = parser.parse(response)

# Convert the parsed listings into a JSON-serializable format, then create a DataFrame with that data.
df = pd.DataFrame(jsonable_encoder(result.listings))

# Display the first few rows of the DataFrame to verify and inspect the results.
df.head()


,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Paris 8th District,1350000,2,2,1076,Beautifully renovated apartment offering 100 s...,"Located on Avenue Montaigne, in the prestigiou..."
1,Paris 5th District,980000,1,1,753,Charming 70 sqm apartment in a historic buildi...,"Situated in the Latin Quarter, close to the So..."
2,Paris 16th District,2650000,4,3,2153,Spacious 200 sqm apartment in a prestigious Ha...,Located in the upscale residential area of Pas...
3,Paris 3rd District,620000,1,1,592,Modern loft-style apartment with 55 sqm of liv...,"Situated in the trendy Marais district, known ..."
4,Paris 7th District,1785000,3,2,1453,Elegant 135 sqm apartment in a classic Haussma...,Located near the Champ de Mars and the Eiffel ...


In [14]:
# Display all the rows of the DataFrame to verify and inspect the results.
df.head(n=10)

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Paris 8th District,1350000,2,2,1076,Beautifully renovated apartment offering 100 s...,"Located on Avenue Montaigne, in the prestigiou..."
1,Paris 5th District,980000,1,1,753,Charming 70 sqm apartment in a historic buildi...,"Situated in the Latin Quarter, close to the So..."
2,Paris 16th District,2650000,4,3,2153,Spacious 200 sqm apartment in a prestigious Ha...,Located in the upscale residential area of Pas...
3,Paris 3rd District,620000,1,1,592,Modern loft-style apartment with 55 sqm of liv...,"Situated in the trendy Marais district, known ..."
4,Paris 7th District,1785000,3,2,1453,Elegant 135 sqm apartment in a classic Haussma...,Located near the Champ de Mars and the Eiffel ...
5,Paris 11th District,890000,2,1,861,Renovated 80 sqm apartment with a contemporary...,"Situated in the vibrant Bastille district, kno..."
6,Paris 1st District,1950000,2,2,1288,Luxurious 120 sqm apartment in a historic buil...,"Located in the heart of the city, near the Pal..."
7,Paris 6th District,1450000,3,2,1135,Classic Parisian apartment with 105 sqm of liv...,Situated near the Luxembourg Gardens and famou...
8,Paris 9th District,720000,1,1,682,"Cozy 63 sqm apartment with a modern design, fe...",Located near the Opera Garnier and luxury depa...
9,Paris 2nd District,1050000,2,1,893,Stylish 83 sqm apartment with a contemporary d...,"Situated in the vibrant Montorgueil district, ..."


## Storing the Real Estate Listings in a Vector Database and Generating and Storing Embeddings



In [15]:
def load_listings_from_dataframe(df: pd.DataFrame) -> REListingCollection:
    """
    Convert a pandas DataFrame of real estate listings into an REListingCollection.

    :param df: A pandas DataFrame that contains columns corresponding to the REListing fields.
    :return: An REListingCollection object populated with the listings from the DataFrame.
    """
    listings = []

    # Iterate through each row in the DataFrame.
    for _, row in df.iterrows():
        # Create a single REListing object from the current row's data.
        listing = REListing(
            neighborhood=row["neighborhood"],
            price=int(row["price"]),
            bedrooms=int(row["bedrooms"]),
            bathrooms=int(row["bathrooms"]),
            house_size=int(row["house_size"]),
            description=row["description"],
            neighborhood_description=row["neighborhood_description"],
        )
        # Accumulate the listing in the list.
        listings.append(listing)

    # Wrap all listings into an REListingCollection and return.
    return REListingCollection(listings=listings)

# Example usage: Convert the DataFrame into an REListingCollection object.
csv_collection = load_listings_from_dataframe(df)
print(csv_collection)


listings=[REListing(neighborhood='Paris 8th District', price=1350000, bedrooms=2, bathrooms=2, house_size=1076, description='Beautifully renovated apartment offering 100 sqm of living space with an elegant and contemporary design. The property features a spacious living room, dining area, fully equipped kitchen, two bedrooms, two bathrooms, and a balcony with stunning views of the Eiffel Tower.', neighborhood_description='Located on Avenue Montaigne, in the prestigious Golden Triangle area, known for its luxury shops and elegant architecture.'), REListing(neighborhood='Paris 5th District', price=980000, bedrooms=1, bathrooms=1, house_size=753, description='Charming 70 sqm apartment in a historic building with high ceilings and original parquet flooring. The property includes a large living room, fully equipped kitchen, bedroom, bathroom, and a balcony overlooking a tranquil courtyard.', neighborhood_description='Situated in the Latin Quarter, close to the Sorbonne University and the Pa

In [16]:
## Generate text to embed

def build_text_for_embedding(listing: REListing) -> str:
    """
    Create a string representation of an REListing object
    that can be used for embedding (e.g., for vector search).
    """
    return (
        f"Neighborhood: {listing.neighborhood}\n"
        f"Price: {listing.price}\n"
        f"Size: {listing.house_size}\n"
        f"Bedrooms: {listing.bedrooms}\n"
        f"Bathrooms: {listing.bathrooms}\n"
        f"Description: {listing.description}\n\n"
        f"Neighborhood Description: {listing.neighborhood_description}"
    )

# Example usage: Builds a text string from the first listing in csv_collection,
# which can be fed into an embedding model or stored as needed.
build_text_for_embedding(csv_collection.listings[0])

'Neighborhood: Paris 8th District\nPrice: 1350000\nSize: 1076\nBedrooms: 2\nBathrooms: 2\nDescription: Beautifully renovated apartment offering 100 sqm of living space with an elegant and contemporary design. The property features a spacious living room, dining area, fully equipped kitchen, two bedrooms, two bathrooms, and a balcony with stunning views of the Eiffel Tower.\n\nNeighborhood Description: Located on Avenue Montaigne, in the prestigious Golden Triangle area, known for its luxury shops and elegant architecture.'

In [17]:
# Use Pydantic to define a data model that aligns with LanceDB’s requirements.
# This model includes a 'vector' field to store embeddings (size 1536),
# along with additional fields for real estate details.

from lancedb.pydantic import vector, LanceModel

class RE_listing(LanceModel):
    vector: vector(1536)  # Stores the embedding vector representation of the listing.
    Neighborhood: str
    Price: int
    Size: int
    Bedrooms: int
    Bathrooms: int
    Description: str
    Neighborhood_Description: str


In [18]:
## Generate Embeddings

import openai
import os

# Configure the OpenAI API key from environment variables.
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_embedding(text: str) -> list[float]:
    """
    Use OpenAI’s Embedding API to generate a vector representation (embedding)
    for the input text using the 'text-embedding-ada-002' model.
    """
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )

    # The embedding is found in the 'data' field of the response (index 0).
    return response["data"][0]["embedding"]

data = []
# Iterate over each listing in our collection.
for listing in csv_collection.listings:
    # Generate an embedding for the text representation of the listing.
    embed = generate_embedding(build_text_for_embedding(listing))

    # Print out the listing text and its corresponding embedding for debugging/inspection.
    print(build_text_for_embedding(listing))
    print("Embeddings:", embed)
    print("Embedding length:", len(embed))

    # Append a new RE_listing object to 'data', populating both fields for metadata
    # and the embedding vector.
    data.append(
        RE_listing(
            vector=embed,
            Neighborhood=listing.neighborhood,
            Size=listing.house_size,
            Price=listing.price,
            Bedrooms=listing.bedrooms,
            Bathrooms=listing.bathrooms,
            Description=listing.description,
            Neighborhood_Description=listing.neighborhood_description
        )
    )
    print("")

Neighborhood: Paris 8th District
Price: 1350000
Size: 1076
Bedrooms: 2
Bathrooms: 2
Description: Beautifully renovated apartment offering 100 sqm of living space with an elegant and contemporary design. The property features a spacious living room, dining area, fully equipped kitchen, two bedrooms, two bathrooms, and a balcony with stunning views of the Eiffel Tower.

Neighborhood Description: Located on Avenue Montaigne, in the prestigious Golden Triangle area, known for its luxury shops and elegant architecture.
Embeddings: [0.001362750306725502, 0.021406402811408043, -0.005495891906321049, -0.005656215362250805, 0.016186267137527466, 0.018020369112491608, -0.015121718868613243, -0.0087857311591506, -0.03991415351629257, -0.00549909844994545, -0.0068490225821733475, 0.021637268364429474, -0.009478328749537468, -0.02097032219171524, -0.024215271696448326, -0.005871049128472805, 0.014570205472409725, 0.008926816284656525, -0.007387709803879261, -0.002957969671115279, -0.021342273801565

In [19]:
# Now connect to a local db at ~/.lancedb and create an empty LanceDB table

import lancedb

# Connect to LanceDB, specifying the local path where data will be stored.
db = lancedb.connect("~/.lancedb")

# Define the name of the table that will store real estate listings.
table_name = "RE_listing"

# Drop the table if it already exists, ignoring the error if it doesn’t.
db.drop_table(table_name, ignore_missing=True)

# Create a new table with the schema defined by the RE_listing Pydantic model.
table = db.create_table(table_name, schema=RE_listing)

In [20]:
# Insert the records into the LanceDB table.
# Each 'd' in 'data' is an RE_listing object, so we convert it into a dictionary
# before passing it to the 'table.add()' method for insertion.
table.add([dict(d) for d in data])


In [21]:
# Preview the first 10 rows of the LanceDB table by converting them to a Pandas DataFrame.
table.head(n=10).to_pandas()


,vector,Neighborhood,Price,Size,Bedrooms,Bathrooms,Description,Neighborhood_Description
0,"[0.0013627503, 0.021406403, -0.005495892, -0.0...",Paris 8th District,1350000,1076,2,2,Beautifully renovated apartment offering 100 s...,"Located on Avenue Montaigne, in the prestigiou..."
1,"[0.008876329, 0.02562266, -0.009469805, -0.008...",Paris 5th District,980000,753,1,1,Charming 70 sqm apartment in a historic buildi...,"Situated in the Latin Quarter, close to the So..."
2,"[-0.004162335, 0.032003008, 0.0003227024, -0.0...",Paris 16th District,2650000,2153,4,3,Spacious 200 sqm apartment in a prestigious Ha...,Located in the upscale residential area of Pas...
3,"[0.008733928, 0.029100297, -0.007121511, -0.01...",Paris 3rd District,620000,592,1,1,Modern loft-style apartment with 55 sqm of liv...,"Situated in the trendy Marais district, known ..."
4,"[-0.00015916913, 0.038581494, -0.010954929, -0...",Paris 7th District,1785000,1453,3,2,Elegant 135 sqm apartment in a classic Haussma...,Located near the Champ de Mars and the Eiffel ...
5,"[0.007984041, 0.027314845, -0.0075515998, -0.0...",Paris 11th District,890000,861,2,1,Renovated 80 sqm apartment with a contemporary...,"Situated in the vibrant Bastille district, kno..."
6,"[-0.0040557324, 0.030966327, -0.0075858417, -0...",Paris 1st District,1950000,1288,2,2,Luxurious 120 sqm apartment in a historic buil...,"Located in the heart of the city, near the Pal..."
7,"[0.008281393, 0.03468955, 0.0052816314, -0.015...",Paris 6th District,1450000,1135,3,2,Classic Parisian apartment with 105 sqm of liv...,Situated near the Luxembourg Gardens and famou...
8,"[0.010928544, 0.029936958, 0.012857872, -0.006...",Paris 9th District,720000,682,1,1,"Cozy 63 sqm apartment with a modern design, fe...",Located near the Opera Garnier and luxury depa...
9,"[0.005265168, 0.035396524, -0.0048416085, -0.0...",Paris 2nd District,1050000,893,2,1,Stylish 83 sqm apartment with a contemporary d...,"Situated in the vibrant Montorgueil district, ..."


## Testing Semantic Search of Listings Based on Buyer Preferences

In [22]:
# Define a user query that describes the type of property they want.
user_query = "Looking for a 1-bedroom apartment with a balcony"

# Generate an embedding vector for the user’s query using the same model as before.
query_embed = generate_embedding(user_query)

# Perform a similarity search in the LanceDB table using the query embedding.
# Limit the results to the top 3 most similar listings, and convert the results to a Pandas DataFrame.
results = table.search(query_embed).limit(3).to_pandas()

# Print out the search results for inspection.
print(results)


                                              vector         Neighborhood  \
0  [0.007984041, 0.027314845, -0.0075515998, -0.0...  Paris 11th District   
1  [0.010928544, 0.029936958, 0.012857872, -0.006...   Paris 9th District   
2  [0.008876329, 0.02562266, -0.009469805, -0.008...   Paris 5th District   

    Price  Size  Bedrooms  Bathrooms  \
0  890000   861         2          1   
1  720000   682         1          1   
2  980000   753         1          1   

                                         Description  \
0  Renovated 80 sqm apartment with a contemporary...   
1  Cozy 63 sqm apartment with a modern design, fe...   
2  Charming 70 sqm apartment in a historic buildi...   

                            Neighborhood_Description  _distance  
0  Situated in the vibrant Bastille district, kno...   0.339351  
1  Located near the Opera Garnier and luxury depa...   0.343874  
2  Situated in the Latin Quarter, close to the So...   0.347549  


## Building the User Preference Interface

In [23]:
# hard-code the buyer preferences in questions and answers
questions = [
                "How big do you want your house to be?"
                "What are 3 most important things for you in choosing this property?",
                "Which amenities would you like?",
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",
            ]

In [24]:
# Helper function to get the top K listings based on a list of user preferences.
def get_top_listings(preferences: list[str], top_k: int=5) -> list[int]:
    '''
    Given a list of preference strings, this function will:
    1. Combine them into a single query string.
    2. Generate an embedding for that query.
    3. Use similarity search against the LanceDB table.
    4. Return the top K listings' data (neighborhood, price, etc.).
    '''
    top_listingsP = []
    top_listingsBed = []
    top_listingsBath = []
    top_listingsSize = []
    top_listingsDesc = []
    top_listingsNeigh = []
    top_listingsNeighDesc = []

    # Combine the user's multiple preferences into a single query string.
    combined_pref = '\n'.join(preferences)

    # Generate an embedding for the combined preferences.
    query_embed = generate_embedding(combined_pref)

    # Search for the top K most similar listings and convert results to a DataFrame.
    results = table.search(query_embed).limit(top_k).to_pandas()

    # Iterate through the top results, appending each relevant field to the lists.
    for index, row in results.iterrows():
        top_listingsNeigh.append(row['Neighborhood'])
        top_listingsP.append(row['Price'])
        top_listingsSize.append(row['Size'])
        top_listingsBed.append(row['Bedrooms'])
        top_listingsBath.append(row['Bathrooms'])
        top_listingsDesc.append(row['Description'])
        top_listingsNeighDesc.append(row['Neighborhood_Description'])

        # Stop if we've reached the requested top_k listings.
        if len(top_listingsNeigh) == top_k:
            break

    # Return each list in an organized format for further processing.
    return top_listingsNeigh, top_listingsP, top_listingsBed, top_listingsBath, top_listingsSize, top_listingsDesc, top_listingsNeighDesc

In [25]:
# Hardcode the buyer's preferences in ANSWERS. This list simulates the user's requirements.
ANSWERS = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "access to urban amenities like restaurants and theaters."
]

In [26]:
# Call the get_top_listings function with the user's preferences and capture the returned data.
resDesc, resP, resBed, resBath, resSize, resNeig, resNeigDesc = get_top_listings(ANSWERS)

In [27]:
# Print the description of the top listing.
print("Description:", resDesc[0])

# Print the price of the top listing.
print("Price:", resP[0])

# Print the number of bedrooms in the top listing.
print("#Bedrooms:", resBed[0])

# Print the number of bathrooms in the top listing.
print("#Bathrooms:", resBath[0])

# Print the size (in square feet) of the top listing.
print("Size:", resSize[0])

# Print the neighborhood where the top listing is located.
print("Neighborhood:", resNeig[0])

# Print a description of the neighborhood for the top listing.
print("Neighborhood Description:", resNeigDesc[0])

Description: Paris 6th District
Price: 1450000
#Bedrooms: 3
#Bathrooms: 2
Size: 1135
Neighborhood: Classic Parisian apartment with 105 sqm of living space, featuring a spacious living room, dining area, fully equipped kitchen, three bedrooms, two bathrooms, and a balcony with views of the historic Saint-Germain-des-Prés neighborhood.
Neighborhood Description: Situated near the Luxembourg Gardens and famous cafes such as Café de Flore, in a chic and intellectual district.


## LLM Augmentation

In [28]:
# Construct a formatted string containing details of the top real estate listing.
# This context will be used to provide the language model with specific information
# about the property when generating an augmented description.
context = (
    "Description: {}\n"
    "Price: {}\n"
    "#Bedrooms: {}\n"
    "#Bathrooms: {}\n"
    "Size: {}\n"
    "Neighborhood: {}\n"
    "Neighborhood Description: {}"
).format(
    resDesc[0],          # Description of the property
    resP[0],             # Price in USD
    resBed[0],           # Number of bedrooms
    resBath[0],          # Number of bathrooms
    resSize[0],          # Size in square feet
    resNeig[0],          # Neighborhood name
    resNeigDesc[0]       # Description of the neighborhood
)

# Assign the user's query to the variable 'question'.
# This query represents what the user is asking about the property.
question = user_query

# Create an augmentation prompt template that combines the context and the user's question.
# The prompt instructs the language model to:
# 1. Provide an answer to the user's question based on the given context.
# 2. Augment the property's description to better align with the buyer’s specific preferences.
#    This involves highlighting features of the property that match the buyer's desires.
augmentation_prompt_template = (
    f"Based on the following context:\n{context}\n----\n"
    f"Provide an answer to the question '{question}' and also augment the description, "
    f"tailoring it to resonate with the buyer’s specific preferences. This involves "
    f"subtly emphasizing aspects of the property that align with what the buyer is looking for."
)

# Print the augmentation prompt template.
# This is useful for debugging purposes to ensure that the prompt is correctly formatted
# before sending it to the language model.
print(augmentation_prompt_template)

Based on the following context:
Description: Paris 6th District
Price: 1450000
#Bedrooms: 3
#Bathrooms: 2
Size: 1135
Neighborhood: Classic Parisian apartment with 105 sqm of living space, featuring a spacious living room, dining area, fully equipped kitchen, three bedrooms, two bathrooms, and a balcony with views of the historic Saint-Germain-des-Prés neighborhood.
Neighborhood Description: Situated near the Luxembourg Gardens and famous cafes such as Café de Flore, in a chic and intellectual district.
----
Provide an answer to the question 'Looking for a 1-bedroom apartment with a balcony' and also augment the description, tailoring it to resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.


In [29]:
# Function to call the OpenAI GPT-3.5 API
def generate_augmentation(prompt_template):
    """
    Generates an augmented description based on the provided prompt template
    by interacting with OpenAI's GPT-3.5-turbo model.

    :param prompt_template: A formatted string containing the context and instructions
                            for the language model.
    :return: The generated augmented description or an error message if the API call fails.
    """
    try:
        # Calling the OpenAI ChatCompletion API with a system message and the user prompt.
        # Note: The method to call may vary based on the OpenAI library version:
        # - Use openai.ChatCompletion.create for OpenAI library versions < 1.0
        # - Use openai.chat.completions.create for OpenAI library versions >= 1.0
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Specifies the model to use.
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a real estate expert. You are writing the matching property "
                        "according to the buyer's criteria and you want to close the deal but "
                        "you want to be accurate, honest and factual."
                    )
                },
                {
                    "role": "user",
                    "content": prompt_template  # The user's prompt containing context and instructions.
                }
            ],
            temperature=1,          # Controls the randomness of the output. Higher values produce more creative responses.
            max_tokens=256,         # The maximum number of tokens to generate in the response.
            top_p=1,                # Nucleus sampling parameter. 1 means no restriction.
            frequency_penalty=0,    # Penalizes new tokens based on their existing frequency in the text so far.
            presence_penalty=0      # Penalizes new tokens based on whether they appear in the text so far.
        )

        # Extract and return the content of the generated message from the response.
        # The response structure contains a list of choices; we take the first one.
        return response.choices[0].message.content

    except Exception as e:
        # If an error occurs during the API call, return an error message.
        return f"An error occurred: {e}"

# Generating the response from the model using the augmentation prompt template.
ans = generate_augmentation(augmentation_prompt_template)

# Printing the augmented description received from the language model.
print("Augmented:")
print(ans)

Augmented:
Dear Buyer,

I understand that you are looking for a 1-bedroom apartment with a balcony, and I believe I have the perfect property that aligns with your preferences.

Although the apartment I have available features 3 bedrooms, it also boasts a balcony overlooking the picturesque Saint-Germain-des-Prés neighborhood. The balcony provides a perfect space for enjoying your morning coffee or evening unwind as you take in the charming surroundings.

This classic Parisian apartment offers 105 sqm of living space, a spacious living room, a dining area, a fully equipped kitchen, and two bathrooms. The property is situated in a chic and intellectual district near the Luxembourg Gardens and renowned cafes like Café de Flore, offering a blend of tranquility and vibrant city life.

I believe this property provides the ideal balance of charm, location, and amenities that you are seeking. I would love to show it to you in person and help you envision the potential this apartment holds for